In [1]:
import numpy as np
import pandas as pd

from helper.loader import load_ratings, load_movies, load_lists, load_500_1000_corr
from helper.preprocesser import preprocess_ratings, prepare_test_data_dense
from helper.application import application
import helper.collaborative as coll
import helper.showResults as sh

from tqdm import tqdm

# Performance

Notebook to check the performance of our different models by predicting ratings which we manually removed to have a ground truth

In [2]:
ratings = load_ratings()
movies = load_movies()
lists = load_lists()

In [3]:
ratings_new, lists_new = preprocess_ratings(ratings, lists, 500,1000)
dense_user_item = coll.get_dense_user_item(ratings_new)
popu_matrix = coll.get_popularity(lists_new, dense_user_item)

average_ratings = coll.compute_average_ratings(dense_user_item)

In [4]:
sum(dense_user_item.count())

3188952

starting with only 1/1000 of data missing, this represent 3188 ratings to predict

## 10 data

In [5]:
train_data,train_label = prepare_test_data_dense(dense_user_item,1/300000)

100%|██████████| 10/10 [00:00<00:00, 8890.00it/s]


In [1]:
#train_label

In [2]:
#dense_user_item

In [3]:
#key1,key2 = list(train_label.keys())[0]


In [4]:
#dense_user_item.columns[key2]

In [6]:
# Each removed ratings is for a new user -> will have to compute a lot of prediction, maybe something cleaner but didn't see

classic_norm_SSE = 0
classic_SSE = 0
hybrid_norm_SSE = 0
hybrid_SSE = 0
pop_norm_SSE = 0
pop_SSE = 0

rounded_classic_norm_error = 0
rounded_classic_error = 0
rounded_hybrid_norm_error = 0
rounded_hybrid_error = 0
rounded_pop_norm_error = 0
rounded_pop_error = 0


for key1,key2 in tqdm(train_label.keys()):

    user = train_data.index[key1]
    item = train_data.columns[key2]

    similarity = coll.get_k_dynamic_similar_users(train_data,user,40)
    popularity = coll.get_k_popularity(popu_matrix,similarity)

    ##################
    weight_classic = similarity
    
    pred_norm = coll.predict_value_norm(user,item,weight_classic,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_classic,train_data,average_ratings)

    classic_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    classic_SSE += (pred - train_label[(key1,key2)])**2

    rounded_classic_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_classic_error += abs(round(pred) - train_label[(key1,key2)])

    ##################
    hybrid = similarity.loc[user] + popularity.loc["popularity"]
    weight_hybrid = hybrid.to_frame().transpose() 

    pred_norm = coll.predict_value_norm(user,item,weight_hybrid,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_hybrid,train_data,average_ratings)

    hybrid_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    hybrid_SSE += (pred - train_label[(key1,key2)])**2

    rounded_hybrid_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_hybrid_error += abs(round(pred) - train_label[(key1,key2)])

    ##################
    weight_pop = popularity

    pred_norm = coll.predict_value_norm(user,item,weight_pop,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_pop,train_data,average_ratings)
        
    pop_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    pop_SSE += (pred - train_label[(key1,key2)])**2

    rounded_pop_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_pop_error += abs(round(pred) - train_label[(key1,key2)])
    ##################



100%|██████████| 10/10 [00:07<00:00,  1.34it/s]


In [7]:
nbr = len(train_label)

print("For classical model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(classic_norm_SSE/nbr,rounded_classic_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(classic_SSE/nbr,rounded_classic_error/nbr))

print("For Hybrid model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(hybrid_norm_SSE/nbr,rounded_hybrid_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(hybrid_SSE/nbr,rounded_hybrid_error/nbr))

print("For Popularity only based model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(pop_norm_SSE/nbr,rounded_pop_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(pop_SSE/nbr,rounded_pop_error/nbr))

For classical model: 
With normalization: average SSE = 0.20546733649878557, average rounded diff = 0.3
Without normalization: average SSE = 0.18302398249376867, average rounded diff = 0.2
For Hybrid model: 
With normalization: average SSE = 0.18888464924174314, average rounded diff = 0.3
Without normalization: average SSE = 0.17094301720658348, average rounded diff = 0.2
For Popularity only based model: 
With normalization: average SSE = 0.12329947094744798, average rounded diff = 0.2
Without normalization: average SSE = 0.34396025560666804, average rounded diff = 0.4


## 30 data

In [8]:
train_data,train_label = prepare_test_data_dense(dense_user_item,1/100000)

####################
classic_norm_SSE = 0
classic_SSE = 0
hybrid_norm_SSE = 0
hybrid_SSE = 0
pop_norm_SSE = 0
pop_SSE = 0

rounded_classic_norm_error = 0
rounded_classic_error = 0
rounded_hybrid_norm_error = 0
rounded_hybrid_error = 0
rounded_pop_norm_error = 0
rounded_pop_error = 0


for key1,key2 in tqdm(train_label.keys()):

    user = train_data.index[key1]
    item = train_data.columns[key2]

    similarity = coll.get_k_dynamic_similar_users(train_data,user,40)
    popularity = coll.get_k_popularity(popu_matrix,similarity)

    ##################
    weight_classic = similarity
    
    pred_norm = coll.predict_value_norm(user,item,weight_classic,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_classic,train_data,average_ratings)

    classic_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    classic_SSE += (pred - train_label[(key1,key2)])**2

    rounded_classic_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_classic_error += abs(round(pred) - train_label[(key1,key2)])

    ##################
    hybrid = similarity.loc[user] + popularity.loc["popularity"]
    weight_hybrid = hybrid.to_frame().transpose() 

    pred_norm = coll.predict_value_norm(user,item,weight_hybrid,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_hybrid,train_data,average_ratings)

    hybrid_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    hybrid_SSE += (pred - train_label[(key1,key2)])**2

    rounded_hybrid_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_hybrid_error += abs(round(pred) - train_label[(key1,key2)])

    ##################
    weight_pop = popularity

    pred_norm = coll.predict_value_norm(user,item,weight_pop,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_pop,train_data,average_ratings)
        
    pop_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    pop_SSE += (pred - train_label[(key1,key2)])**2

    rounded_pop_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_pop_error += abs(round(pred) - train_label[(key1,key2)])
    ##################



##################################
nbr = len(train_label)

print("For classical model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(classic_norm_SSE/nbr,rounded_classic_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(classic_SSE/nbr,rounded_classic_error/nbr))

print("For Hybrid model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(hybrid_norm_SSE/nbr,rounded_hybrid_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(hybrid_SSE/nbr,rounded_hybrid_error/nbr))

print("For Popularity only based model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(pop_norm_SSE/nbr,rounded_pop_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(pop_SSE/nbr,rounded_pop_error/nbr))

100%|██████████| 31/31 [00:24<00:00,  1.28it/s]

For classical model: 
With normalization: average SSE = 0.8581998990867361, average rounded diff = 0.7419354838709677
Without normalization: average SSE = 1.1175007472736047, average rounded diff = 0.7741935483870968
For Hybrid model: 
With normalization: average SSE = 0.8915128001507262, average rounded diff = 0.7741935483870968
Without normalization: average SSE = 1.1264048736707912, average rounded diff = 0.8064516129032258
For Popularity only based model: 
With normalization: average SSE = 1.0320785799561667, average rounded diff = 0.7419354838709677
Without normalization: average SSE = 1.242506356928144, average rounded diff = 0.9032258064516129


## 300 data

In [9]:
train_data,train_label = prepare_test_data_dense(dense_user_item,1/10000)

####################
classic_norm_SSE = 0
classic_SSE = 0
hybrid_norm_SSE = 0
hybrid_SSE = 0
pop_norm_SSE = 0
pop_SSE = 0

rounded_classic_norm_error = 0
rounded_classic_error = 0
rounded_hybrid_norm_error = 0
rounded_hybrid_error = 0
rounded_pop_norm_error = 0
rounded_pop_error = 0


for key1,key2 in tqdm(train_label.keys()):

    user = train_data.index[key1]
    item = train_data.columns[key2]

    similarity = coll.get_k_dynamic_similar_users(train_data,user,40)
    popularity = coll.get_k_popularity(popu_matrix,similarity)

    ##################
    weight_classic = similarity
    
    pred_norm = coll.predict_value_norm(user,item,weight_classic,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_classic,train_data,average_ratings)

    classic_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    classic_SSE += (pred - train_label[(key1,key2)])**2

    rounded_classic_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_classic_error += abs(round(pred) - train_label[(key1,key2)])

    ##################
    hybrid = similarity.loc[user] + popularity.loc["popularity"]
    weight_hybrid = hybrid.to_frame().transpose() 

    pred_norm = coll.predict_value_norm(user,item,weight_hybrid,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_hybrid,train_data,average_ratings)

    hybrid_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    hybrid_SSE += (pred - train_label[(key1,key2)])**2

    rounded_hybrid_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_hybrid_error += abs(round(pred) - train_label[(key1,key2)])

    ##################
    weight_pop = popularity

    pred_norm = coll.predict_value_norm(user,item,weight_pop,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_pop,train_data,average_ratings)
        
    pop_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    pop_SSE += (pred - train_label[(key1,key2)])**2

    rounded_pop_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_pop_error += abs(round(pred) - train_label[(key1,key2)])
    ##################



##################################
nbr = len(train_label)

print("For classical model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(classic_norm_SSE/nbr,rounded_classic_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(classic_SSE/nbr,rounded_classic_error/nbr))

print("For Hybrid model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(hybrid_norm_SSE/nbr,rounded_hybrid_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(hybrid_SSE/nbr,rounded_hybrid_error/nbr))

print("For Popularity only based model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(pop_norm_SSE/nbr,rounded_pop_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(pop_SSE/nbr,rounded_pop_error/nbr))

100%|██████████| 318/318 [03:46<00:00,  1.41it/s]

For classical model: 
With normalization: average SSE = 0.7112991706439104, average rounded diff = 0.6540880503144654
Without normalization: average SSE = 0.7927014635308044, average rounded diff = 0.6509433962264151
For Hybrid model: 
With normalization: average SSE = 0.7178691268824478, average rounded diff = 0.6477987421383647
Without normalization: average SSE = 0.804285097995739, average rounded diff = 0.6792452830188679
For Popularity only based model: 
With normalization: average SSE = 0.939116679942665, average rounded diff = 0.7327044025157232
Without normalization: average SSE = 1.0880041268901652, average rounded diff = 0.7830188679245284


## 3'000 data

In [10]:
train_data,train_label = prepare_test_data_dense(dense_user_item,1/1000)

####################
classic_norm_SSE = 0
classic_SSE = 0
hybrid_norm_SSE = 0
hybrid_SSE = 0
pop_norm_SSE = 0
pop_SSE = 0

rounded_classic_norm_error = 0
rounded_classic_error = 0
rounded_hybrid_norm_error = 0
rounded_hybrid_error = 0
rounded_pop_norm_error = 0
rounded_pop_error = 0


for key1,key2 in tqdm(train_label.keys()):

    user = train_data.index[key1]
    item = train_data.columns[key2]

    similarity = coll.get_k_dynamic_similar_users(train_data,user,40)
    popularity = coll.get_k_popularity(popu_matrix,similarity)

    ##################
    weight_classic = similarity
    
    pred_norm = coll.predict_value_norm(user,item,weight_classic,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_classic,train_data,average_ratings)

    classic_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    classic_SSE += (pred - train_label[(key1,key2)])**2

    rounded_classic_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_classic_error += abs(round(pred) - train_label[(key1,key2)])

    ##################
    hybrid = similarity.loc[user] + popularity.loc["popularity"]
    weight_hybrid = hybrid.to_frame().transpose() 

    pred_norm = coll.predict_value_norm(user,item,weight_hybrid,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_hybrid,train_data,average_ratings)

    hybrid_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    hybrid_SSE += (pred - train_label[(key1,key2)])**2

    rounded_hybrid_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_hybrid_error += abs(round(pred) - train_label[(key1,key2)])

    ##################
    weight_pop = popularity

    pred_norm = coll.predict_value_norm(user,item,weight_pop,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_pop,train_data,average_ratings)
        
    pop_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    pop_SSE += (pred - train_label[(key1,key2)])**2

    rounded_pop_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_pop_error += abs(round(pred) - train_label[(key1,key2)])
    ##################



##################################
nbr = len(train_label)

print("For classical model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(classic_norm_SSE/nbr,rounded_classic_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(classic_SSE/nbr,rounded_classic_error/nbr))

print("For Hybrid model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(hybrid_norm_SSE/nbr,rounded_hybrid_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(hybrid_SSE/nbr,rounded_hybrid_error/nbr))

print("For Popularity only based model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(pop_norm_SSE/nbr,rounded_pop_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(pop_SSE/nbr,rounded_pop_error/nbr))

100%|██████████| 3188/3188 [37:11<00:00,  1.43it/s]

For classical model: 
With normalization: average SSE = 0.7031799656614639, average rounded diff = 0.6031994981179423
Without normalization: average SSE = 0.7890107127694462, average rounded diff = 0.6558971141781681
For Hybrid model: 
With normalization: average SSE = 0.7138551456235214, average rounded diff = 0.6126097867001254
Without normalization: average SSE = 0.7997318753692487, average rounded diff = 0.6577791718946048
For Popularity only based model: 
With normalization: average SSE = 0.909063123812653, average rounded diff = 0.6985570890840652
Without normalization: average SSE = 1.042783065090846, average rounded diff = 0.7728983688833124


## 10'000 data missing

In [11]:
train_data,train_label = prepare_test_data_dense(dense_user_item,1/300)

####################
classic_norm_SSE = 0
classic_SSE = 0
hybrid_norm_SSE = 0
hybrid_SSE = 0
pop_norm_SSE = 0
pop_SSE = 0

rounded_classic_norm_error = 0
rounded_classic_error = 0
rounded_hybrid_norm_error = 0
rounded_hybrid_error = 0
rounded_pop_norm_error = 0
rounded_pop_error = 0


for key1,key2 in tqdm(train_label.keys()):

    user = train_data.index[key1]
    item = train_data.columns[key2]

    similarity = coll.get_k_dynamic_similar_users(train_data,user,40)
    popularity = coll.get_k_popularity(popu_matrix,similarity)

    ##################
    weight_classic = similarity
    
    pred_norm = coll.predict_value_norm(user,item,weight_classic,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_classic,train_data,average_ratings)

    classic_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    classic_SSE += (pred - train_label[(key1,key2)])**2

    rounded_classic_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_classic_error += abs(round(pred) - train_label[(key1,key2)])

    ##################
    hybrid = similarity.loc[user] + popularity.loc["popularity"]
    weight_hybrid = hybrid.to_frame().transpose() 

    pred_norm = coll.predict_value_norm(user,item,weight_hybrid,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_hybrid,train_data,average_ratings)

    hybrid_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    hybrid_SSE += (pred - train_label[(key1,key2)])**2

    rounded_hybrid_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_hybrid_error += abs(round(pred) - train_label[(key1,key2)])

    ##################
    weight_pop = popularity

    pred_norm = coll.predict_value_norm(user,item,weight_pop,train_data,average_ratings)
    pred = coll.predict_value(user,item,weight_pop,train_data,average_ratings)
        
    pop_norm_SSE += (pred_norm - train_label[(key1,key2)])**2
    pop_SSE += (pred - train_label[(key1,key2)])**2

    rounded_pop_norm_error += abs(round(pred_norm) - train_label[(key1,key2)])
    rounded_pop_error += abs(round(pred) - train_label[(key1,key2)])
    ##################



##################################
nbr = len(train_label)

print("For classical model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(classic_norm_SSE/nbr,rounded_classic_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(classic_SSE/nbr,rounded_classic_error/nbr))

print("For Hybrid model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(hybrid_norm_SSE/nbr,rounded_hybrid_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(hybrid_SSE/nbr,rounded_hybrid_error/nbr))

print("For Popularity only based model: ")
print("With normalization: average SSE = {}, average rounded diff = {}".format(pop_norm_SSE/nbr,rounded_pop_norm_error/nbr))
print("Without normalization: average SSE = {}, average rounded diff = {}".format(pop_SSE/nbr,rounded_pop_error/nbr))

 45%|████▌     | 14486/31889 [2:47:41<3:21:27,  1.44it/s]


KeyboardInterrupt: 

Plotting everything for 10 removed ratings just to see

In [12]:
train_data_2,train_label_2 = prepare_test_data_dense(dense_user_item,1/300000)

100%|██████████| 10/10 [00:00<00:00, 9414.82it/s]


In [15]:
for key1,key2 in train_label_2.keys():

    user = train_data_2.index[key1]
    item = train_data_2.columns[key2]

    similarity = coll.get_k_dynamic_similar_users(train_data_2,user,40)
    popularity = coll.get_k_popularity(popu_matrix,similarity)

    ##################
    weight_classic = similarity
    
    cla_pred_norm = coll.predict_value_norm(user,item,weight_classic,train_data_2,average_ratings)
    cla_pred = coll.predict_value(user,item,weight_classic,train_data_2,average_ratings)

    ##################
    hybrid = similarity.loc[user] + popularity.loc["popularity"]
    weight_hybrid = hybrid.to_frame().transpose() 

    hyb_pred_norm = coll.predict_value_norm(user,item,weight_hybrid,train_data_2,average_ratings)
    hyb_pred = coll.predict_value(user,item,weight_hybrid,train_data_2,average_ratings)

    ##################
    weight_pop = popularity

    pop_pred_norm = coll.predict_value_norm(user,item,weight_pop,train_data_2,average_ratings)
    pop_pred = coll.predict_value(user,item,weight_pop,train_data_2,average_ratings)
    ##################

    print("For user {} and movie {}, the true ratings was {}".format(user,item,train_label_2[(key1,key2)]))
    print("The classical model predicted {} with normalization and {} without".format(cla_pred_norm,cla_pred))
    print("The hybrid model predicted {} with normalization and {} without".format(hyb_pred_norm,hyb_pred))
    print("The popularity model predicted {} with normalization and {} without".format(pop_pred_norm,pop_pred))
    print("################################")

For user 70929901 and movie 2083, the true ratings was 4.0
The classical model predicted 4.366124772998244 with normalization and 4.08230020935102 without
The hybrid model predicted 4.334833886793348 with normalization and 4.036882293536396 without
The popularity model predicted 3.4741589719285715 with normalization and 2.7876348107032083 without
################################
For user 10040458 and movie 402, the true ratings was 4.0
The classical model predicted 5 with normalization and 4.237505533225593 without
The hybrid model predicted 4.889349607306551 with normalization and 4.234722687670517 without
The popularity model predicted 4.579815126019368 with normalization and 4.227625664078463 without
################################
For user 33246005 and movie 4124, the true ratings was 3.0
The classical model predicted 2.2888367633349596 with normalization and 2.584491556826118 without
The hybrid model predicted 2.2888367633349596 with normalization and 2.584491556826118 without
Th